# Setup

In [84]:
import os,re,time
from nltk import word_tokenize,sent_tokenize
from nltk.stem import PorterStemmer,LancasterStemmer

"""please uncomment and run these if your machine does not have needed nltk data"""
# nltk.download('punkt') 
# nltk.download('stopwords')

'please uncomment and run these if your machine does not have needed nltk data'

### Basic info retrival about corpora

In [8]:
# total document files in our corpus
len(os.listdir('english-corpora'))

8635

In [15]:
# all of them are .txt files
all('.txt' in t for t in os.listdir('english-corpora'))

True

In [16]:
os.listdir('english-corpora')[:5]

['D00521.txt', 'P_1709.txt', 'D00920.txt', 'T00143.txt', 'P03064.txt']

In [58]:
english_corpora_files=os.listdir('english-corpora')

### Tasks to do

- [x] TODO: how to remove HTML info from these pages
- [x] TODO: how to use porter's stemmer
- [x] TODO: how to do tokenization
- [x] TODO: how to clean text files: remove non-ascii and what else?; and when removing non-ascii should I remove just one character or whole word
- [x] TODO: don't remove stopwords; at least for tf-idf it does not matter; for binary and BM25??

In [85]:
test_corpora_dir='test-corpora'

In [98]:
def create_folder(folder_name):
    '''
    create folder with name given
    '''
    try:
        os.mkdir(folder_name)
    except FileExistsError:
        pass

def corpora_processor(corpora_files_dir,processed_corpora_dir):
    '''
    Input: corpora files dir; processed folder name

    Output: processed corpora files
    '''
    # TODO: add stemmer specifing functionality
    create_folder(processed_corpora_dir)

    porter=PorterStemmer()

    for file in os.listdir(corpora_files_dir):
        with open(corpora_files_dir+'/'+file,'r') as f:
            corpora_file_str=f.read() # text str

            """remove css code lines"""
            css_regex=re.compile(r'.mw.*}')
            # substitue regex expression by ''
            corpora_file_str=css_regex.sub('',corpora_file_str)

            """remove html tag lines"""
            html_regex=re.compile(r'<.*>')
            # substitue regex expression by ''
            corpora_file_str=html_regex.sub('',corpora_file_str)

            """tokenize file str"""
            # here each doc could be processed line by line; i am doing whole doc str
            tokens=word_tokenize(corpora_file_str)
            
            """stem file tokens"""
            processed_corpora_file_str=''
            for token in tokens:
                # avoid non-ascii as suggested in the question
                if token.isascii():
                    processed_corpora_file_str+=porter.stem(token)
                    processed_corpora_file_str+=' ' # to seperate processed tokens

        """store processed file with same name"""
        with open(processed_corpora_dir+'/'+file,'w') as output_file:
            output_file.write(processed_corpora_file_str)

- time taken for whole corpora is too much. around 1600 sec or 25 min
    - one solution is to directly stem file string without tokenization; so we stem whole string together rather than each token and after stemming we tokenize and store; it could save half time

In [99]:
# testing processor on test-corpora of 100 files
corpora_processor(test_corpora_dir,'processed-test-corpora')

---

In [107]:
# tokenization of a file withput using word_tokenizer from nltk
with open('test-corpora/C00001.txt','r') as doc:
    file=doc.read()
    regex_exp3=re.compile(r'\n') # to sub \n by ' '
    regex_exp4=re.compile(r'\t') # to sub \n by ''
    clean_text=regex_exp3.sub(' ',file)
    clean_text=regex_exp4.sub('',clean_text)
    tokens=clean_text.split(' ')
    print(len(tokens))

10640
